# Tagoloan Data Mining Notebook

<div>
    <img src='https://riverbasin.denr.gov.ph/img/Tagoloan%20RB/TagoloanRB3.jpg' width='500px'/>
</div>

This notebook processes the cleaned data from the data cleaning notebook. Processes of this notebook includes the following:
1. Removes hour and minute from the timestamp.
2. Clean duplicated data by keeping the first duplicated variable.
3. Ammend missing Water Level Data by Interpolation.
4. File integrity checker

### Authors
- Gifrey John M. Sulay

### Checked by:
- Dr. Anabel A. Abuzo
- Engr. Augustine Ave Padagunan

# CODE PROPER

### Import Modules and Foo Data

In [1]:
import numpy as np
import pandas as pd
import re

__author__ = "Gifrey John M. Sulay"
__copyright__ = "Xavier University - Engineering Resource Center"

saved_data_path = 'Cleaned_Tagoloan'

### Open Dataset

In [2]:
#open data
tag_path='Edited_Data/tag.csv'
rg0_path='Edited_Data/rg0.csv'
rg1_path='Edited_Data/rg1.csv'

tag=pd.read_csv(tag_path)
rg1=pd.read_csv(rg1_path)
rg0=pd.read_csv(rg0_path)

In [3]:
tag.head(3)

,Unnamed: 0,Station Name,Sensor Type,Sensor Value,Sensor ID,Units,Timestamp,Sensor Label
0,0,FGEN Tagoloan Gauge Station,Level,1.654815,46481,Meters,12/2/2019 12:47:00 AM,Water Level
1,1,FGEN Tagoloan Gauge Station,Level,1.678690,46481,Meters,12/2/2019 1:47:00 AM,Water Level
2,2,FGEN Tagoloan Gauge Station,Level,1.654815,46481,Meters,12/2/2019 2:47:00 AM,Water Level


In [4]:
rg0.head(3)

,Unnamed: 0,Station Name,Sensor Type,Sensor Value,Sensor ID,Units,Timestamp,Sensor Label
0,0,FGEN Tagoloan Gauge Station,Rain,207.2,46491,mm,1/27/2020 12:47:00 AM,Rain Gauge
1,1,FGEN Tagoloan Gauge Station,Rain,207.2,46491,mm,1/27/2020 1:47:00 AM,Rain Gauge
2,2,FGEN Tagoloan Gauge Station,Rain,207.2,46491,mm,1/27/2020 2:47:00 AM,Rain Gauge


In [5]:
rg1.head(3)

,Unnamed: 0,Station Name,Sensor Type,Sensor Value,Sensor ID,Units,Timestamp,Sensor Label
0,0,FGEN Tagoloan Gauge Station,DailyRain,0.112,1546480,mm,10/1/2020 12:59:59 AM,Rain Gauge 1
1,1,FGEN Tagoloan Gauge Station,DailyRain,0.112,1546480,mm,10/1/2020 1:59:59 AM,Rain Gauge 1
2,2,FGEN Tagoloan Gauge Station,DailyRain,0.112,1546480,mm,10/1/2020 2:59:59 AM,Rain Gauge 1


### Cleaning
This section does the following
1. Acquires *Timestamp* and *Sensor Value* data from cleaned data set.
2. Removes minute and seconds in *Timestamp* column.
3. Creates dataframe of duplicated values.

In [6]:
#filter data
tag_f=tag[['Timestamp','Sensor Value']].rename(columns={'Sensor Value':'Water_Level'})
rg0_f=rg0[['Timestamp','Sensor Value']].rename(columns={'Sensor Value':'RG0_Level'})
rg1_f=rg1[['Timestamp','Sensor Value']].rename(columns={'Sensor Value':'RG1_Level'})

In [7]:
def timestamp_conv(timestamp):
    timestamp=timestamp.split()
    date=timestamp[0]
    time=timestamp[1]
    hms=time.split(':')
    if len(timestamp)==3:
        morn_aft=timestamp[2]
        if int(hms[0])==12 and morn_aft=='AM':
            hour=0
        elif int(hms[0])==12 and morn_aft=='PM':
            hour=12
        elif morn_aft=='PM':
            hour=int(hms[0])+12
        elif morn_aft=='AM':
            hour=int(hms[0])

    if len(timestamp)==2:
        hour=int(hms[0])
    #reorganized timestamp to y/m/d
    date_split=date.split('/')
    month=date_split[0]
    day=date_split[1]
    year=date_split[2]
    reorganized_date=f"{int(year)}/{int(month)}/{int(day)}"
    new_timestamp=f"{reorganized_date} {int(hour)}:00:00"
    return new_timestamp

In [8]:
#convert timestamps
tag_n = tag_f.copy()
tag_n['Timestamp']=tag_n['Timestamp'].apply(timestamp_conv)
rg0_n = rg0_f.copy()
rg0_n['Timestamp']=rg0_n['Timestamp'].apply(timestamp_conv)
rg1_n = rg1_f.copy()
rg1_n['Timestamp']=rg1_n['Timestamp'].apply(timestamp_conv)

In [9]:
tag_n.to_csv('tag_n_tbr.csv')

In [10]:
#generate timestamp array for timestamp index for dataframe
#returns clean_timestamp

start="2021-08-01"
end="2021-10-01"
import datetime
x = pd.date_range(start=start, end=end, freq='H').tolist()
clean_timestamp=[]
for i in x:
    month=i.strftime("%m")
    day=i.strftime("%d")
    year=i.strftime("%Y")
    hour=i.strftime("%H")
    
    string=f"{int(year)}/{int(month)}/{int(day)} {int(hour)}:00:00"
    clean_timestamp.append(string)

In [11]:
def duplicates_df(df):
    try:
        new_df = pd.concat([g for _, g in df.groupby("Timestamp") if len(g) > 1])
        return new_df
    except:
        new_df = df.iloc[0:0]
        return new_df
#create df of duplicated values
tag_n_duplicates=duplicates_df(tag_n)
rg0_n_duplicates=duplicates_df(rg0_n)
rg1_n_duplicates=duplicates_df(rg1_n)

#drop redundant data points
tag_n.drop_duplicates(subset='Timestamp', keep=False, inplace=True)
rg0_n.drop_duplicates(subset='Timestamp', keep=False, inplace=True)
rg1_n.drop_duplicates(subset='Timestamp', keep=False, inplace=True)

### Merging
This section accomplishes the following:
1. Merges Water Level to Each Rain Gauge

In [12]:
#create dataframe for complete and correct timestamp
ts=pd.DataFrame({'Timestamp':clean_timestamp})

#create base dataframe(timestamp and water level)
ts_tag_n=pd.merge(ts,tag_n,how='left',on='Timestamp')
base=ts_tag_n

In [13]:
#merge base and rain gauge 0
base_rg0=pd.merge(base,rg0_n,how='left',on='Timestamp')
base_rg0.loc[base_rg0['RG0_Level'].isnull()]

,Timestamp,Water_Level,RG0_Level
14,2021/8/1 14:00:00,NaN,NaN
15,2021/8/1 15:00:00,NaN,NaN
16,2021/8/1 16:00:00,NaN,NaN
24,2021/8/2 0:00:00,NaN,NaN
28,2021/8/2 4:00:00,NaN,NaN
...,...,...,...
1460,2021/9/30 20:00:00,3.158940,NaN
1461,2021/9/30 21:00:00,2.920190,NaN
1462,2021/9/30 22:00:00,2.944065,NaN
1463,2021/9/30 23:00:00,2.776940,NaN


In [14]:
#merge base and rain gauge 1
base_rg1=pd.merge(base,rg1_n,how='left',on='Timestamp')
base_rg1

,Timestamp,Water_Level,RG1_Level
0,2021/8/1 0:00:00,1.798065,0.104
1,2021/8/1 1:00:00,1.845815,0.104
2,2021/8/1 2:00:00,1.869690,0.104
3,2021/8/1 3:00:00,1.798065,0.104
4,2021/8/1 4:00:00,1.798065,0.104
...,...,...,...
1460,2021/9/30 20:00:00,3.158940,0.136
1461,2021/9/30 21:00:00,2.920190,0.136
1462,2021/9/30 22:00:00,2.944065,0.136
1463,2021/9/30 23:00:00,2.776940,0.136


### First Save
This section does the following:
1. Saves merged datas of each rain gauge to **Cleaned_Data.xlsx**
2. Saves dataframe of missing values to **Missing Values.xlsx**

In [15]:
#save to excel sheet
with pd.ExcelWriter(f'{saved_data_path}/Cleaned_Data.xlsx') as writer:
    base_rg0.to_excel(writer, sheet_name='Water Level - Rain Gauge 0')
    base_rg1.to_excel(writer, sheet_name='Water Level - Rain Gauge 1')

In [16]:
#create spreadsheet of missing values
water_level_missing=base_rg0[base_rg0['Water_Level'].isnull()].index.tolist()
rg0_missing=base_rg0[base_rg0['RG0_Level'].isnull()].index.tolist()
rg1_missing=base_rg1[base_rg1['RG1_Level'].isnull()].index.tolist()

water_level_missing_df=pd.DataFrame({'Missing Water Level':water_level_missing})
rg0_missing_df=pd.DataFrame({'Missing RG0':rg0_missing})
rg1_missing_df=pd.DataFrame({'Missing RG1':rg1_missing})

with pd.ExcelWriter(f'{saved_data_path}/Missing Values.xlsx') as writer:
    water_level_missing_df.to_excel(writer, sheet_name='Water Level')
    rg0_missing_df.to_excel(writer, sheet_name='Rain Gauge 0')
    rg1_missing_df.to_excel(writer, sheet_name='Rain Gauge 1')
    
with pd.ExcelWriter(f'{saved_data_path}/Duplicated Values.xlsx') as writer:
    tag_n_duplicates.set_index('Timestamp').to_excel(writer, sheet_name='Water Level')
    rg0_n_duplicates.set_index('Timestamp').to_excel(writer, sheet_name='Rain Gauge 0')
    rg1_n_duplicates.set_index('Timestamp').to_excel(writer, sheet_name='Rain Gauge 1')

### Ammendment of Missing Values
This section does the following
1. Water Level
    - Interpolate missing values using np.interpolate's linear method.
2. Rain Gauge 0
    - Replace *None* values in Rain Gauge 0 using np.interpolate's padding method.
3. Saves the ammended data in **Corrected_Data.xlsx**.

**Note:**
Data given in Rain Gauge 1 is unclear whether the data given is correct. *Thus, Rain Gauge 1 will not be used in the succeeding analysis.*

### Rain Gauge 0

In [17]:
#interpolate water level
new_water_level=base['Water_Level'].astype('float64').interpolate()

#replace nan values in RG0 using np.interpolate's padding method
rg0_copy=base_rg0['RG0_Level'].astype('float64').interpolate()

#create difference on Rain Gauge 0
rg0_diff=[0]
count=1
for i in rg0_copy[1:]:
    diff=i-rg0_copy[count-1]
    rg0_diff.append(diff)
    count+=1
    
rg0_copy=base_rg0['RG0_Level'].astype('float64').interpolate()

#create dataframe with corrected data using base dataframe
corrected_df=base
corrected_df['Corrected_Water_Level']= new_water_level
corrected_df=corrected_df.drop(columns=['Water_Level'])
corrected_df['Corrected_RG0_Level']=rg0_copy
corrected_df['RG0_Diff']=rg0_diff
corrected_df=corrected_df.reset_index()

def hours_only(df):
    timestamp_arr=df['Timestamp']
    hour_lst=[]
    for i in timestamp_arr:
        time=i.split()[1]
        hour=time.split(':')[0]
        hour_lst.append(hour)
    df['Hour']=hour_lst
    return df

corrected_df=hours_only(corrected_df)
headers=corrected_df.columns.values.tolist()
headers.remove('Hour')
headers.insert(0,'Hour')
corrected_df=corrected_df[headers]
corrected_df0 = corrected_df
corrected_df0

,Hour,index,Timestamp,Corrected_Water_Level,Corrected_RG0_Level,RG0_Diff
0,0,0,2021/8/1 0:00:00,1.798065,252.8,0.0
1,1,1,2021/8/1 1:00:00,1.845815,252.8,0.0
2,2,2,2021/8/1 2:00:00,1.869690,252.8,0.0
3,3,3,2021/8/1 3:00:00,1.798065,252.8,0.0
4,4,4,2021/8/1 4:00:00,1.798065,252.8,0.0
...,...,...,...,...,...,...
1460,20,1460,2021/9/30 20:00:00,3.158940,391.4,0.0
1461,21,1461,2021/9/30 21:00:00,2.920190,391.4,0.0
1462,22,1462,2021/9/30 22:00:00,2.944065,391.4,0.0
1463,23,1463,2021/9/30 23:00:00,2.776940,391.4,0.0


In [29]:
base_rg1.loc[base_rg1['Water_Level'].isnull()]

,Timestamp,Water_Level,RG1_Level
14,2021/8/1 14:00:00,NaN,0.000
15,2021/8/1 15:00:00,NaN,NaN
16,2021/8/1 16:00:00,NaN,NaN
24,2021/8/2 0:00:00,NaN,0.000
28,2021/8/2 4:00:00,NaN,0.000
...,...,...,...
1120,2021/9/16 16:00:00,NaN,0.000
1144,2021/9/17 16:00:00,NaN,0.144
1160,2021/9/18 8:00:00,NaN,NaN
1255,2021/9/22 7:00:00,NaN,2.112


### Rain Gauge 1

In [19]:
#interpolate water level
new_water_level=base['Water_Level'].astype('float64').interpolate()

#replace nan values in RG1 using np.interpolate's padding method
rg1_copy=base_rg1['RG1_Level'].astype('float64').interpolate()

#create difference on Rain Gauge 0
rg1_diff=[0]
count=1
for i in rg1_copy[1:]:
    diff=i-rg1_copy[count-1]
    rg1_diff.append(diff)
    count+=1
rg1_copy=base_rg1['RG1_Level'].astype('float64').interpolate()

#create dataframe with corrected data using base dataframe
corrected_df=base
corrected_df['Corrected_Water_Level']= new_water_level
corrected_df=corrected_df.drop(columns=['Water_Level'])
corrected_df['Corrected_RG1_Level']=rg1_copy
corrected_df['RG1_Diff']=rg1_diff
corrected_df=corrected_df.reset_index()

def hours_only(df):
    timestamp_arr=df['Timestamp']
    hour_lst=[]
    for i in timestamp_arr:
        time=i.split()[1]
        hour=time.split(':')[0]
        hour_lst.append(hour)
    df['Hour']=hour_lst
    return df

corrected_df=hours_only(corrected_df)
headers=corrected_df.columns.values.tolist()
headers.remove('Hour')
headers.insert(0,'Hour')
corrected_df=corrected_df[headers]
corrected_df1 = corrected_df
corrected_df1

,Hour,index,Timestamp,Corrected_Water_Level,Corrected_RG1_Level,RG1_Diff
0,0,0,2021/8/1 0:00:00,1.798065,0.104,0.0
1,1,1,2021/8/1 1:00:00,1.845815,0.104,0.0
2,2,2,2021/8/1 2:00:00,1.869690,0.104,0.0
3,3,3,2021/8/1 3:00:00,1.798065,0.104,0.0
4,4,4,2021/8/1 4:00:00,1.798065,0.104,0.0
...,...,...,...,...,...,...
1460,20,1460,2021/9/30 20:00:00,3.158940,0.136,0.0
1461,21,1461,2021/9/30 21:00:00,2.920190,0.136,0.0
1462,22,1462,2021/9/30 22:00:00,2.944065,0.136,0.0
1463,23,1463,2021/9/30 23:00:00,2.776940,0.136,0.0


In [20]:
with pd.ExcelWriter(f'{saved_data_path}/Corrected_Data.xlsx') as writer:
    corrected_df0.to_excel(writer,sheet_name='Corrected_Water_and_RG0_Level')
    corrected_df1.to_excel(writer,sheet_name='Corrected_Water_and_RG1_Level')

### Dataframe Integrity Check
Checks if dataframe has missing values

In [21]:
def check_integrity(df):
    return df[df.isnull().any(axis=1)]
x=check_integrity(corrected_df)
type(x)
x

,Hour,index,Timestamp,Corrected_Water_Level,Corrected_RG1_Level,RG1_Diff


### Ammendment 1: Erroneous Water Level
Erronoeous water level data were found. Data are catergorized as erroneous if the difference between the previous and current datapoint exceeds the stated difference limite(diff_limit)

Objective:
* Create array and collect velues larger than stated difference limit
* Remove erroneous data
* Interpolate removed values


In [22]:
def clean_water_level(corrected_df):
    water_level = corrected_df['Corrected_Water_Level']
    #diff limit determines whether the difference is erroneous if difference is more than diff limit then data is erroneous
    diff_limit = 0.5
    is_erroneous=[0]
    count = 1
    for i in water_level[1:]:
        val = i-water_level[count - 1]
        val = abs(val)
        if val >= diff_limit:
            is_erroneous.append(1)
        else:
            is_erroneous.append(0)
        count += 1
        
    corrected_df['err_check']=is_erroneous
    corrected_df.loc[corrected_df['err_check']==1, 'Corrected_Water_Level'] = np.nan
    
    edited_water_level=corrected_df.loc[:, 'Corrected_Water_Level']
    final_water_level=edited_water_level.astype('float64').interpolate()
    corrected_df=corrected_df.drop(columns=['Corrected_Water_Level','err_check'])
    corrected_df['Corrected_Water_Level']=final_water_level
    return corrected_df

In [23]:
corrected_df0 = clean_water_level(corrected_df0)
corrected_df1 = clean_water_level(corrected_df1)

In [24]:
with pd.ExcelWriter(f'{saved_data_path}/Corrected_Data.xlsx') as writer:
    corrected_df0.to_excel(writer,sheet_name='Corrected_Water_and_RG0_Level')
    corrected_df1.to_excel(writer,sheet_name='Corrected_Water_and_RG1_Level')

### Ammendment 2: Daily Water Count
Create daily where per day max water level and cumulated rain gauge data is added to new data frame

In [25]:
daily_df_timestamp = []
count = 1
for i in corrected_df.index.values.tolist():
    if count%24 != 0:
        count += 1
        continue
    else:
        date=i.split()[0]
        daily_df_timestamp.append(date)
    count += 1

AttributeError: 'int' object has no attribute 'split'

In [ ]:
daily_df_rain_gauge = []
count=1
temp = []
for i in corrected_df.RG0_Diff:
    if count % 24 != 0:
        temp.append(i)
        count += 1
    else:
        total = 0
        for n in temp:
            total = total + n
        daily_df_rain_gauge.append(total)
        temp = []
        temp.append(i)
        count += 1

In [ ]:
daily_water_level = []
count=1
temp = []
for i in corrected_df.Corrected_Water_Level:
    if count % 24 != 0:
        temp.append(i)
        count += 1
    else:
        total = 0
        for n in temp:
            if total < n:
                total = n
            else:
                continue
        daily_water_level.append(total)
        temp = []
        temp.append(i)
        count += 1



In [ ]:
daily_df = pd.DataFrame({'Timestamp':daily_df_timestamp,'Water_Level':daily_water_level,'RG0_Diff':daily_df_rain_gauge})
daily_df=daily_df.set_index('Timestamp')

In [ ]:
daily_df

In [ ]:
with pd.ExcelWriter(f'{saved_data_path}/Daily_Data.xlsx') as writer:
    daily_df.to_excel(writer,sheet_name='Sheet 1')